## run notebooks

In [30]:
%run "../Notebooks/intialization.ipynb"
%run "../Notebooks/Stitching_functions.ipynb"
%run "../Notebooks/Visualization_functions.ipynb"
%run "../Notebooks/Downsampling_and_outliers_functions.ipynb"
%load_ext autoreload

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [31]:
## show files in directory recenlty modified
#!find . -mtime 0

## Load the pointclouds

In [32]:
## parameters initialization 
myparams = "./ScreenCamera_2021-01-12-10-28-18.json"  # parameter for camera point view, json file via pressing P
myconfiguration_file = "RenderOption_2021-01-12-10-24-27.json" # configuration file for properties, json file via pressing o


In [33]:
stitched_external_pc = o3d.io.read_point_cloud("stitched_external_pc.pcd")
stitched_internal_pc = o3d.io.read_point_cloud("stitched_internal_pc.pcd")
stitched_upper_pc = o3d.io.read_point_cloud("stitched_upper_pc.pcd")

In [34]:
labels_stitches = ["stitched_external_pc","stitched_internal_pc", "stitched_upper_pc"]
list_stitches = [stitched_external_pc,stitched_internal_pc,stitched_upper_pc]
color_stitches = [[1, 0.706, 0],[0, 0.651, 0.929],[0,0.7,0.3]]

## Many plots

In [35]:
# plot a list of geometries
custom_draw_geometry([list_stitches[:]],
                     mytitle = [labels_stitches[:]],
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized


### downsampling

In [36]:
downsampled_stitches = downsample_list_stitches(list_stitches,
                                                #labels_stitches,color_stitches , 
                                                voxel_size = 0.3,
                                                visualization_on = True,
                                                print_statements = True)

Downsampled_pcl_1
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_1
number of points original :  230502
number of points with voxel down sample : 16013

Downsampled_pcl_2
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_2
number of points original :  227230
number of points with voxel down sample : 11910

Downsampled_pcl_3
loading parameters: 
  ./ScreenCamera_2021-01-12-10-28-18.json
loading configuration file: 
  RenderOption_2021-01-12-10-24-27.json
[Open3D WARNING] GLFW Error: The GLFW library is not initialized
Downsampled_pcl_3
number of points original :  219648
number of points with voxel down sample : 13335



In [24]:
## alignment check
#i = 0
#o3d.visualization.draw_geometries([downsampled_stitches[i].paint_uniform_color([0.5, 0, 0]),
                                   #list_stitches[i].paint_uniform_color([0, 0.5, 0])])

## Merge the external, upper, internal together

### Step 1- 

In [39]:
# timestamp
now = datetime.now()
dt_string = now.strftime("%d%m%Y-%Hh%Mm%Ss")
print(dt_string)

# initialize 
voxel_size  =0.2 # with 0.1 doesnt work!
i = 2
j = 0
source_label = labels_stitches[i][9:-3] # just labels parsing
target_label =labels_stitches[j][9:-3]

source = list_stitches[i]
target = list_stitches[j]

processed_source = downsampled_stitches[i]
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)
start = timer()

#MAIN
source_output, target_output, transformation_1 = stitch_sequences(processed_source,processed_target,
                                                  source_label,target_label,
                                                  mmax_iteration = 10**7,
                                                  mmax_validation = 0.999,
                                                  print_statements = True, 
                                                  visualization_on = False  
                                                 )

end = timer()
print (f"duration: {end-start}")
draw_registration_result(source_output, target_output, transformation_1, title = dt_string+"Downsampled Registration results")


## apply transformation on original 
draw_registration_result(source, target, transformation_1, title = dt_string+"Full Registration results")

12012021-13h10m17s
allpc_downsampled_upper_external
Load two point clouds and disturb initial pose.
removing outliers

GLOBAL REGISTRATION: RANSAC registration on downsampled point clouds.
Transformation is:
[[ 9.89051385e-01 -1.00081331e-03  1.47568138e-01 -6.95234796e+00]
 [-6.46001788e-02  8.96137313e-01  4.39049807e-01 -2.54936752e+01]
 [-1.32680722e-01 -4.43775748e-01  8.86261198e-01  7.67879794e+00]
 [ 0.00000000e+00  0.00000000e+00  0.00000000e+00  1.00000000e+00]]
overlapping points :  20
RegistrationResult with fitness=5.735589e-03, inlier_rmse=2.087349e-01, and correspondence_set size of 20
Access transformation to get result.

POINT-TO-PLANE ICP registration is applied on original point
distance threshold 0.400.
Transformation is:
[[  0.98770938  -0.03760983   0.15170921  -7.10811636]
 [ -0.05060687   0.8413838    0.53806342 -30.1652896 ]
 [ -0.14788215  -0.53912782   0.82913935  10.51694402]
 [  0.           0.           0.           1.        ]]
overlapping points :  3210


In [29]:
# timestamp
now = datetime.now()
dt_string = now.strftime("%d%m%Y-%Hh%Mm%Ss")
print(dt_string)

# initialize 
voxel_size  =0.2 # with 0.1 doesnt work!
i = 2
j = 0
source_label = labels_stitches[i][9:-3] # just labels parsing
target_label =labels_stitches[j][9:-3]

source = list_stitches[i]
target = list_stitches[j]

processed_source = downsampled_stitches[i]
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)
start = timer()

#MAIN
source_output, target_output, transformation_1 = stitch_sequences(processed_source,processed_target,
                                                  source_label,target_label,
                                                  mmax_iteration = 10**8,
                                                  mmax_validation = 0.999,
                                                  print_statements = True, 
                                                  visualization_on = False  
                                                 )

end = timer()
print (f"duration: {end-start}")
draw_registration_result(source_output, target_output, transformation_1, title = dt_string+"Downsampled Registration results")


## apply transformation on original 
draw_registration_result(source, target, transformation_1, title = dt_string+"Full Registration results")

12012021-12h49m52s
allpc_downsampled_upper_external
Load two point clouds and disturb initial pose.
removing outliers

GLOBAL REGISTRATION: RANSAC registration on downsampled point clouds.
Transformation is:
[[ -0.04969377   0.13012126   0.99025198  -6.22751835]
 [  0.82299035   0.56708341  -0.0332159  -31.57271881]
 [ -0.56587756   0.8133172   -0.13526906  11.07439226]
 [  0.           0.           0.           1.        ]]
overlapping points :  31
RegistrationResult with fitness=8.910606e-03, inlier_rmse=2.148738e-01, and correspondence_set size of 31
Access transformation to get result.

POINT-TO-PLANE ICP registration is applied on original point
distance threshold 0.400.
Transformation is:
[[ -0.03762227   0.15169919   0.98771044  -7.10767312]
 [  0.84138228   0.53806725  -0.0505914  -30.16567625]
 [ -0.53912932   0.8291387   -0.14788032  10.51691426]
 [  0.           0.           0.           1.        ]]
overlapping points :  3210
RegistrationResult with fitness=2.407199e-01, in

## downsample new one for next step

In [ ]:
downsampled_upper_external = downsample_list_stitches([upper_external],
                                                        ["upper_external"],color_stitches= None , 
                                                        voxel_size = 0.3,
                                                        visualization_on = False,
                                                        print_statements = False)

## Step 2-

In [ ]:
# input 
voxel_size  =0.2
j = 1
source_label = "upper_external"
target_label =labels_stitches[j][9:-3]

processed_source = upper_external
processed_target = downsampled_stitches[j]

mytitle= dt_string+"-%s_%s"%(source_label,target_label)


start = timer()

source_output, target_output,transformation = tqdm(stitch_sequences(processed_source,processed_target,
                                                                  source_label,target_label,
                                                                  mmax_iteration = 10**7,
                                                                  mmax_validation = 0.999,
                                                                  print_statements = False,
                                                                  visualization_on = False
                                                                 )
                                                )


end = timer()
print (f"duration: {end-start}")
draw_registration_result(source_output, target_output, transformation, title = "Downsampled Registration results")

In [ ]:
#source = upper_external
#target = list_stitches[j]

draw_registration_result(source, target, transformation, title = "Registration results")



In [ ]:
source, target,transformation = tqdm(stitch_sequences(processed_source,processed_target,
                                                                  source_label,target_label,
                                                                  mmax_iteration = 10**7,
                                                                  mmax_validation = 0.999,
                                                                  print_statements = False,
                                                                  visualization_on = False
                                                                 )
                                                )

## visualize result

In [ ]:
#draw_registration_result(source, target, transformation, title = "")

In [ ]:
"""
complete_downsampled = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = False,
                                        visualize_result = False)
"""

In [ ]:
#get_num_points([complete_downsampled], print_statement = True)

## apply transformation on original 

In [ ]:
#use the found transformation with original full point source and target

source = upper_external
target = list_stitches[j]

draw_registration_result(source, target, transformation, title = "")

In [ ]:
mytitle

In [ ]:
#save it
complete = save_registration_result(source, target, transformation, 
                                        title= mytitle, 
                                        save_result = True,
                                        visualize_result = False)

custom_draw_geometry(complete,
                     mytitle = mytitle,
                     params = myparams,  # parameter for camera point view, json file via pressing P
                     configuration_file = myconfiguration_file, # configuration file for properties, json file via pressing o
                     take_screen_shot = False,
                     rotate = True)

In [ ]:
get_num_points([complete], print_statement = True)